# 🧮 **01 - Embeddings: Base Vetorial Fitness**

## 🎯 **Objetivo:**
Configurar sistema de embeddings e processar a base de conhecimento fitness.

## 📋 **O que faremos:**
1. ⚙️ Setup dos embeddings com SentenceTransformers
2. 📄 Chunking da base de conhecimento
3. 🧮 Geração de embeddings vetoriais
4. ✅ Testes de similaridade semântica

---

## 1️⃣ **Setup e Verificação de Dependências**

In [1]:
# 📦 VERIFICAÇÃO E IMPORTAÇÃO DE BIBLIOTECAS

import sys
import importlib
from pathlib import Path
import numpy as np
import json
import hashlib
from typing import Dict, List, Any, Tuple

def verificar_instalacao(pacote):
    """Verifica se um pacote está instalado"""
    try:
        importlib.import_module(pacote)
        return True
    except ImportError:
        return False

# Pacotes essenciais para embeddings
pacotes_embeddings = {
    'sentence_transformers': 'sentence_transformers',
    'numpy': 'numpy',
    'scikit_learn': 'sklearn'
}

print("🔍 Verificando dependências para embeddings...")
todos_ok = True
for nome_display, nome_modulo in pacotes_embeddings.items():
    status = verificar_instalacao(nome_modulo)
    emoji = "✅" if status else "❌"
    print(f"{emoji} {nome_display}")
    if not status:
        todos_ok = False

if todos_ok:
    print("\n🎉 Todas as dependências estão instaladas!")
else:
    print("\n💡 Execute: pip install sentence-transformers scikit-learn")

print(f"\n📋 Status: {'✅ PRONTO' if todos_ok else '⚠️ VERIFICAR INSTALAÇÃO'}")

🔍 Verificando dependências para embeddings...
✅ sentence_transformers
✅ numpy
✅ scikit_learn

🎉 Todas as dependências estão instaladas!

📋 Status: ✅ PRONTO


## 2️⃣ **Carregamento da Base de Conhecimento**

In [2]:
# 📄 CARREGAMENTO DA BASE FITNESS

def carregar_base_conhecimento():
    """Carrega a base de conhecimento fitness do arquivo"""
    arquivo_base = Path("base_conhecimento_fitness.txt")
    
    if not arquivo_base.exists():
        print("⚠️ Arquivo base_conhecimento_fitness.txt não encontrado!")
        return criar_base_demo()
    
    try:
        with open(arquivo_base, 'r', encoding='utf-8') as f:
            conteudo = f.read()
        print(f"✅ Base carregada: {arquivo_base.stat().st_size} bytes")
        return conteudo
    except Exception as e:
        print(f"❌ Erro ao carregar: {e}")
        return criar_base_demo()

def criar_base_demo():
    """Cria base mínima para demonstração"""
    return """
### EXERCÍCIOS BÁSICOS

**Supino reto**
Exercício fundamental para peitoral maior, deltóide anterior e tríceps.

**Agachamento**
Rei dos exercícios para quadríceps, glúteos e core.

### HIPERTROFIA
Séries: 3-4
Repetições: 8-12
Descanso: 60-90 segundos
"""

# Carregar base
conhecimento_bruto = carregar_base_conhecimento()
print(f"\n📊 Total de caracteres: {len(conhecimento_bruto)}")
print(f"📝 Primeiras 200 chars: {conhecimento_bruto[:200]}...")

✅ Base carregada: 8822 bytes

📊 Total de caracteres: 8515
📝 Primeiras 200 chars: # BASE DE CONHECIMENTO - FITNESS & MUSCULAÇÃO

## EXERCÍCIOS POR GRUPO MUSCULAR

### PEITO
**Supino reto com barra**
- Músculos: Peitoral maior, deltóide anterior, tríceps
- Execução: Deite no banco, ...


## 3️⃣ **Chunking Inteligente**

In [3]:
# ✂️ SISTEMA DE CHUNKING INTELIGENTE

def fazer_chunking(texto: str, chunk_size: int = 300) -> List[Dict]:
    """Divide texto em chunks preservando contexto semântico"""
    chunks = []
    linhas = texto.split('\n')
    
    chunk_atual = ""
    secao_atual = "Introdução"
    
    for linha in linhas:
        linha = linha.strip()
        
        # Detectar nova seção (### Título)
        if linha.startswith('### '):
            # Salvar chunk anterior
            if chunk_atual.strip():
                chunks.append({
                    'id': len(chunks),
                    'texto': chunk_atual.strip(),
                    'secao': secao_atual,
                    'tokens': len(chunk_atual.split()),
                    'hash': hashlib.md5(chunk_atual.encode()).hexdigest()[:8]
                })
            
            # Nova seção
            secao_atual = linha.replace('### ', '').strip()
            chunk_atual = linha + "\n"
            
        else:
            chunk_atual += linha + "\n"
            
            # Dividir se muito grande
            if len(chunk_atual.split()) > chunk_size:
                chunks.append({
                    'id': len(chunks),
                    'texto': chunk_atual.strip(),
                    'secao': secao_atual,
                    'tokens': len(chunk_atual.split()),
                    'hash': hashlib.md5(chunk_atual.encode()).hexdigest()[:8]
                })
                chunk_atual = ""
    
    # Último chunk
    if chunk_atual.strip():
        chunks.append({
            'id': len(chunks),
            'texto': chunk_atual.strip(),
            'secao': secao_atual,
            'tokens': len(chunk_atual.split()),
            'hash': hashlib.md5(chunk_atual.encode()).hexdigest()[:8]
        })
    
    return chunks

# Processar chunking
print("✂️ Processando chunking...")
chunks = fazer_chunking(conhecimento_bruto)

print(f"\n📊 ESTATÍSTICAS DO CHUNKING:")
print(f"   • Total de chunks: {len(chunks)}")
print(f"   • Tokens médios por chunk: {sum(c['tokens'] for c in chunks) / len(chunks):.1f}")
print(f"   • Seções identificadas: {len(set(c['secao'] for c in chunks))}")

# Mostrar alguns chunks
print(f"\n📄 EXEMPLOS DE CHUNKS:")
for i, chunk in enumerate(chunks[:3]):
    print(f"\n   Chunk {i+1} (ID: {chunk['id']}, Hash: {chunk['hash']})")
    print(f"   Seção: {chunk['secao']}")
    print(f"   Tokens: {chunk['tokens']}")
    print(f"   Texto: {chunk['texto'][:100]}...")

✂️ Processando chunking...

📊 ESTATÍSTICAS DO CHUNKING:
   • Total de chunks: 22
   • Tokens médios por chunk: 55.4
   • Seções identificadas: 22

📄 EXEMPLOS DE CHUNKS:

   Chunk 1 (ID: 0, Hash: 01b4d409)
   Seção: Introdução
   Tokens: 13
   Texto: # BASE DE CONHECIMENTO - FITNESS & MUSCULAÇÃO

## EXERCÍCIOS POR GRUPO MUSCULAR...

   Chunk 2 (ID: 1, Hash: 85a251ba)
   Seção: PEITO
   Tokens: 168
   Texto: ### PEITO
**Supino reto com barra**
- Músculos: Peitoral maior, deltóide anterior, tríceps
- Execuçã...

   Chunk 3 (ID: 2, Hash: ca945e8b)
   Seção: COSTAS
   Tokens: 152
   Texto: ### COSTAS
**Puxada frontal**
- Músculos: Grande dorsal, rombóides, bíceps
- Execução: Puxe a barra ...


## 4️⃣ **Geração de Embeddings**

In [4]:
# 🧮 SISTEMA DE EMBEDDINGS

# Importar SentenceTransformers
try:
    from sentence_transformers import SentenceTransformer
    
    print("🔄 Carregando modelo de embeddings...")
    # Modelo multilingual otimizado
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    
    print("✅ Modelo carregado com sucesso!")
    print(f"📏 Dimensão dos embeddings: {embedding_model.get_sentence_embedding_dimension()}")
    
except ImportError:
    print("❌ SentenceTransformers não disponível")
    embedding_model = None
except Exception as e:
    print(f"⚠️ Erro ao carregar modelo: {e}")
    embedding_model = None

🔄 Carregando modelo de embeddings...
✅ Modelo carregado com sucesso!
📏 Dimensão dos embeddings: 384


In [5]:
# 🎯 GERAÇÃO DE EMBEDDINGS VETORIAIS

if embedding_model:
    print("🧮 Gerando embeddings para todos os chunks...")
    
    # Extrair textos dos chunks
    textos = [chunk['texto'] for chunk in chunks]
    
    # Gerar embeddings em batch (mais eficiente)
    embeddings = embedding_model.encode(textos, show_progress_bar=True)
    
    # Adicionar embeddings aos chunks
    for i, chunk in enumerate(chunks):
        chunk['embedding'] = embeddings[i]
        chunk['embedding_norm'] = np.linalg.norm(embeddings[i])  # Norma para referência
    
    print(f"\n✅ Embeddings gerados!")
    print(f"📊 Shape dos embeddings: {embeddings.shape}")
    print(f"📏 Dimensões: {embeddings.shape[1]}")
    print(f"🎯 Norma média: {np.mean([c['embedding_norm'] for c in chunks]):.3f}")
    
else:
    print("⚠️ Pulando geração de embeddings - modelo não disponível")
    embeddings = None

🧮 Gerando embeddings para todos os chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


✅ Embeddings gerados!
📊 Shape dos embeddings: (22, 384)
📏 Dimensões: 384
🎯 Norma média: 1.000


## 5️⃣ **Testes de Similaridade**

In [6]:
# 🔍 FUNÇÃO DE BUSCA POR SIMILARIDADE

def buscar_similares(query: str, chunks: List[Dict], modelo, top_k: int = 3) -> List[Dict]:
    """Busca chunks similares usando similaridade coseno"""
    
    if not modelo or not chunks or 'embedding' not in chunks[0]:
        print("❌ Embeddings não disponíveis - usando busca por palavras")
        return buscar_palavras_chave(query, chunks, top_k)
    
    try:
        # Gerar embedding da query
        query_embedding = modelo.encode([query])[0]
        
        # Calcular similaridades
        similaridades = []
        for chunk in chunks:
            # Similaridade coseno
            sim = np.dot(query_embedding, chunk['embedding']) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(chunk['embedding'])
            )
            similaridades.append((sim, chunk))
        
        # Ordenar por similaridade
        similaridades.sort(key=lambda x: x[0], reverse=True)
        
        # Retornar top-k com scores
        resultados = []
        for sim, chunk in similaridades[:top_k]:
            resultado = chunk.copy()
            resultado['similaridade'] = float(sim)
            resultados.append(resultado)
        
        return resultados
        
    except Exception as e:
        print(f"❌ Erro na busca semântica: {e}")
        return buscar_palavras_chave(query, chunks, top_k)

def buscar_palavras_chave(query: str, chunks: List[Dict], top_k: int = 3) -> List[Dict]:
    """Busca por palavras-chave como fallback"""
    query_lower = query.lower()
    query_words = set(query_lower.split())
    
    resultados = []
    for chunk in chunks:
        texto_lower = chunk['texto'].lower()
        chunk_words = set(texto_lower.split())
        
        # Score baseado em palavras comuns
        palavras_comuns = len(query_words.intersection(chunk_words))
        if palavras_comuns > 0:
            score = palavras_comuns / len(query_words)
            resultado = chunk.copy()
            resultado['similaridade'] = score
            resultados.append(resultado)
    
    # Ordenar por score
    resultados.sort(key=lambda x: x['similaridade'], reverse=True)
    return resultados[:top_k]

print("✅ Funções de busca criadas!")

✅ Funções de busca criadas!


In [7]:
# 🧪 TESTES DE BUSCA SEMÂNTICA

print("🧪 TESTANDO BUSCA SEMÂNTICA")
print("=" * 50)

# Queries de teste
queries_teste = [
    "exercícios para ganhar músculos",
    "treino de peito", 
    "quantas séries fazer",
    "descanso entre exercícios"
]

for i, query in enumerate(queries_teste, 1):
    print(f"\n{i}️⃣ Query: '{query}'")
    print("-" * 30)
    
    resultados = buscar_similares(query, chunks, embedding_model, top_k=2)
    
    for j, resultado in enumerate(resultados, 1):
        print(f"\n   📄 Resultado {j}:")
        print(f"   📊 Similaridade: {resultado['similaridade']:.3f}")
        print(f"   📂 Seção: {resultado['secao']}")
        print(f"   📝 Texto: {resultado['texto'][:120]}...")

print("\n" + "=" * 50)
print("✅ Testes de similaridade concluídos!")

🧪 TESTANDO BUSCA SEMÂNTICA

1️⃣ Query: 'exercícios para ganhar músculos'
------------------------------

   📄 Resultado 1:
   📊 Similaridade: 0.575
   📂 Seção: Introdução
   📝 Texto: # BASE DE CONHECIMENTO - FITNESS & MUSCULAÇÃO

## EXERCÍCIOS POR GRUPO MUSCULAR...

   📄 Resultado 2:
   📊 Similaridade: 0.519
   📂 Seção: PERNAS
   📝 Texto: ### PERNAS

#### QUADRÍCEPS
**Agachamento**
- Músculos: Quadríceps, glúteos, core
- Execução: Descida controlada até 90°...

2️⃣ Query: 'treino de peito'
------------------------------

   📄 Resultado 1:
   📊 Similaridade: 0.462
   📂 Seção: FORÇA
   📝 Texto: ### FORÇA
- **Volume**: Moderado (3-5 séries, 1-6 repetições)
- **Intensidade**: Alta (85-100% 1RM)
- **Frequência**: 3-...

   📄 Resultado 2:
   📊 Similaridade: 0.459
   📂 Seção: CONDICIONAMENTO
   📝 Texto: ### CONDICIONAMENTO
- **Volume**: Alto
- **Intensidade**: Variada
- **Frequência**: 5-6x por semana
- **Modalidades**: H...

3️⃣ Query: 'quantas séries fazer'
------------------------------

 

## 6️⃣ **Persistência dos Dados**

In [8]:
# 💾 SALVAR DADOS PROCESSADOS

def salvar_chunks_processados(chunks: List[Dict], arquivo: str = "fitness_chunks.json"):
    """Salva chunks processados em JSON (sem embeddings)"""
    
    # Preparar dados para salvamento (sem embeddings numpy)
    chunks_para_salvar = []
    for chunk in chunks:
        chunk_limpo = {
            'id': chunk['id'],
            'texto': chunk['texto'],
            'secao': chunk['secao'], 
            'tokens': chunk['tokens'],
            'hash': chunk['hash'],
            'tem_embedding': 'embedding' in chunk
        }
        chunks_para_salvar.append(chunk_limpo)
    
    # Salvar metadata
    dados = {
        'timestamp': str(pd.Timestamp.now()) if 'pd' in globals() else 'unknown',
        'total_chunks': len(chunks),
        'modelo_embedding': 'all-MiniLM-L6-v2' if embedding_model else None,
        'chunks': chunks_para_salvar
    }
    
    try:
        with open(arquivo, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Chunks salvos em {arquivo}")
        print(f"📊 Total: {len(chunks)} chunks")
        
    except Exception as e:
        print(f"❌ Erro ao salvar: {e}")

# Salvar se temos chunks
if chunks:
    salvar_chunks_processados(chunks)
else:
    print("⚠️ Nenhum chunk para salvar")

✅ Chunks salvos em fitness_chunks.json
📊 Total: 22 chunks


## 📊 **Resumo da Etapa 01**

### ✅ **Conquistas:**
- 📄 Base de conhecimento carregada e processada
- ✂️ Chunking inteligente implementado
- 🧮 Embeddings vetoriais gerados
- 🔍 Sistema de busca por similaridade funcional
- 💾 Persistência de dados configurada

---

🎯 **Base vetorial fitness criada com sucesso!** 🏋️‍♂️